# Single module Channel to Channel initial skew 

# Test Description
This script checks the channel-to-channel synchronization by changing the SCLK from 1GHz to 9GHz with fixed jump (can be change) by reading from a Keysight MSO9254A.
This script will compare the skew between Channel 1 and 2 first then Channel 1 and 4 for sampling rate 1GHz to 2.5GHz and then for Channel 1 and channel 3 for sampling rate 1GHz to 9GHz 
The threshold is 50ps for the skew and 15ps for the jitter.

# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable


# Hardware Connection
    Proteus Module    --->     Oscilloscope
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4


# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release - Aurora

In [2]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('..//..//..//SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

In [3]:
# Connect to instrument
inst = connect('192.90.70.22')

# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

#print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000002232810,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


In [4]:
# Build waveforms
seglen = 4096
ncycles = 2
max_dac = 65535
waves = [ None for _ in range(num_channels)]
#Create Square wave
    
for i in range(num_channels):
    cyclelen = seglen / ncycles

    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[i] = y.astype(np.uint16)
    else:
        waves[i] = y.astype(np.uint8)


# Download segments
for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ii +1
    wav = waves[0]
    print('Download wave to segment {} of channel {}'.format(segnum, channb))
    
    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))
    
    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))

    # Select the segment
    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))

    inst.send_scpi_cmd(':VOLT 0.5')
    inst.send_scpi_cmd(':VOLT:OFFS 0')
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)
        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
        
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')
    
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}"')


resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{}"')


Download wave to segment 1 of channel 1
Download wave to segment 2 of channel 2
Download wave to segment 3 of channel 3
Download wave to segment 4 of channel 4


# Skew measurement between channel with same DAC(ie  Channels 1 and 2)

In [5]:
# Osciloscope connection
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))
scope.write(":AUToscale")

*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


12

# Test for skew between channels 
#Connect channel to be compared of the proteus to the channel 1 and 2 of the oscilloscope
#Run the same script for CH1-CH2, CH3-CH4 and CH1-CH3(different DAC)

In [6]:
#CH1-CH2
print('Comparing skew for channel 1 and chanenl 2')
vertical_scale_per_divison = 0.02
horizontal_scale_per_divison = 1e-7
ScopeChannel1,ScopeChannel2 =1,2
# scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write(":AUToscale")
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel1,horizontal_scale_per_divison))
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel2,horizontal_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel1,vertical_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel2,vertical_scale_per_divison))
scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
# Proteus channel 1 and 2
InitialFreqSampleRate,MaximumFreqSampleRate = 1000,2500 #Frequency range #int(input('Maximun sampling rate 2500/9000'))
FreqSampleRateJump = 100 # frequency jump
MAX_SKEW = 50e-12 
# SCOPE_RANGE = 1e-9
scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write('CDIS')
highest_mean=0
freq=0
time.sleep(2)
'''
This loop checks the initial skew between different channels on different module
The maximum skew is 200ps.
Input : Channel 1 , Channel 3 , SampleRateJumps
Output: Pass or Fail and if it fails in which Frequncy in fall
'''
test_success = True
print( 'Initial Skew Test between {} to {} from {}MHz to {}MHz in jumps of {}MHz'.format(ScopeChannel1, ScopeChannel2, InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump)) 
for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):  
    
    inst.send_scpi_cmd(':FREQ:RAST {} MHz'.format(int(FreqSampleRate)))
    time.sleep(1)
    scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
    scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
    scope.write('CDIS')
    scope.write(':MEASure:STATistics ON') 
    scope.write(':MEASure:SENDvalid ON') 
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()

    tmin = float(resualt.split(',')[3])
    tmax = float(resualt.split(',')[4])
    mean = float(resualt.split(',')[5])
    deviation = float(resualt.split(',')[6])
    #pdb.set_trace()
    if (abs(mean)>MAX_SKEW ): #200e-12 ):
        print('Test Failed in Sample clock: {}MHz, Mean (Average): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
        test_success = False
        #break
    else:
	    print('Test Pass in Sample clock: {}MHz, Means(Avrage): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
    if (highest_mean<abs(mean)):
        highest_mean= abs(mean)
        freq= FreqSampleRate
print("***Skew test completed for channel 1 and chaneel 2***")
print('The Highest mean: {} in freq: {} MHz'.format(highest_mean,freq))

Comparing skew for channel 1 and chanenl 2
Initial Skew Test between 1 to 2 from 1000MHz to 2500MHz in jumps of 100MHz


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [ ]:
#CH1-CH4
print('Comparing skew for channel 1 and chanenl 4')
vertical_scale_per_divison = 0.02
horizontal_scale_per_divison = 1e-7
ScopeChannel1,ScopeChannel2 =1,4
# scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel1,horizontal_scale_per_divison))
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel2,horizontal_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel1,vertical_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel2,vertical_scale_per_divison))
scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
# Proteus channel 1 and 2
InitialFreqSampleRate,MaximumFreqSampleRate = 1000,2500 #Frequency range #int(input('Maximun sampling rate 2500/9000'))
FreqSampleRateJump = 100 # frequency jump
MAX_SKEW = 50e-12 
# SCOPE_RANGE = 1e-9
scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write('CDIS')
highest_mean=0
freq=0
time.sleep(2)
'''
This loop checks the initial skew between different channels on different module
The maximum skew is 200ps.
Input : Channel 1 , Channel 3 , SampleRateJumps
Output: Pass or Fail and if it fails in which Frequncy in fall
'''
test_success = True
print( 'Initial Skew Test between {} to {} from {}MHz to {}MHz in jumps of {}MHz'.format(ScopeChannel1, ScopeChannel2, InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump)) 
for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):  
    
    inst.send_scpi_cmd(':FREQ:RAST {} MHz'.format(int(FreqSampleRate)))
    time.sleep(1)
    scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
    scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
    scope.write('CDIS')
    scope.write(':MEASure:STATistics ON') 
    scope.write(':MEASure:SENDvalid ON') 
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()

    tmin = float(resualt.split(',')[3])
    tmax = float(resualt.split(',')[4])
    mean = float(resualt.split(',')[5])
    deviation = float(resualt.split(',')[6])
    #pdb.set_trace()
    if (abs(mean)>MAX_SKEW ): #200e-12 ):
        print('Test Failed in Sample clock: {}MHz, Mean (Average): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
        test_success = False
        #break
    else:
	    print('Test Pass in Sample clock: {}MHz, Means(Avrage): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
    if (highest_mean<abs(mean)):
        highest_mean= abs(mean)
        freq= FreqSampleRate
print("***Skew test completed for channel 1 and chaneel 4***")
print('The Highest mean: {} in freq: {} MHz'.format(highest_mean,freq))

In [8]:
#Channel 1 and 3
print('Comparing skew for channel 1 and chanenl 3')
vertical_scale_per_divison = 0.02
horizontal_scale_per_divison = 1e-7
ScopeChannel1,ScopeChannel2 =1,3
# scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel1,horizontal_scale_per_divison))
scope.write('*RST;:CHAN{0}:DISP ON;:TIMebase:SCALe {1}'.format(ScopeChannel2,horizontal_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel1,vertical_scale_per_divison))
scope.write(':CHAN{0}:SCAL {1};:CHAN{0}:INP DC50'.format(ScopeChannel2,vertical_scale_per_divison))
scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
# Proteus channel 1 and 2
InitialFreqSampleRate,MaximumFreqSampleRate = 1000,9000 #Frequency range #int(input('Maximun sampling rate 2500/9000'))
FreqSampleRateJump = 1000 # frequency jump
MAX_SKEW = 50e-12 
# SCOPE_RANGE = 1e-9
scope.write('*CLS;:DISPlay:CGRade:LEVels ')
scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write('CDIS')
highest_mean=0
freq=0
time.sleep(2)
'''
This loop checks the initial skew between different channels on different module
The maximum skew is 200ps.
Input : Channel 1 , Channel 3 , SampleRateJumps
Output: Pass or Fail and if it fails in which Frequncy in fall
'''
test_success = True
print( 'Initial Skew Test between {} to {} from {}MHz to {}MHz in jumps of {}MHz'.format(ScopeChannel1, ScopeChannel2, InitialFreqSampleRate,MaximumFreqSampleRate,FreqSampleRateJump)) 
for FreqSampleRate in range (InitialFreqSampleRate,MaximumFreqSampleRate+FreqSampleRateJump,FreqSampleRateJump):  
    inst.send_scpi_cmd(':FREQ:RAST 2500e6'.format(int(FreqSampleRate)))
    inst.send_scpi_cmd(':FREQ:RAST {}e6'.format(int(FreqSampleRate)))
    # Download segments
    for ii in range(num_channels):
        ichan = ii
        channb = ichan + 1
        if channb%2!=0:
            segnum = ii +1
            wav = waves[0]
            #print('Download wave to segment {} of channel {}'.format(segnum, channb))

            # Select channel
            inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))

            # Define segment
            inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))

            # Select the segment
            inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))

            inst.send_scpi_cmd(':VOLT 0.5;:VOLT:OFFS 0')
            inst.timeout = 30000

            # Select the segment
            inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))

            # Send the binary-data:
            inst.write_binary_data(':TRAC:DATA', wav)

            resp = inst.send_scpi_query(':SYST:ERR?')
            resp = resp.rstrip()
            if not resp.startswith('0'):
                print('ERROR: "{}" after writing binary values'.format(resp))

            # Play the specified segment at the selected channel:
            cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
            inst.send_scpi_cmd(cmd)

            # Turn on the output of the selected channel:
            inst.send_scpi_cmd(':OUTP ON')

            resp = inst.send_scpi_query(':SYST:ERR?')
            resp = resp.rstrip()
            if not resp.startswith('0'):
                print('ERROR: "{}"')

    
    
    
    
    time.sleep(1)
    scope.write(':SYSTem:HEADer OFF') # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
    scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
    scope.write('CDIS')
    scope.write(':MEASure:STATistics ON') 
    scope.write(':MEASure:SENDvalid ON') 
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()

    tmin = float(resualt.split(',')[3])
    tmax = float(resualt.split(',')[4])
    mean = float(resualt.split(',')[5])
    deviation = float(resualt.split(',')[6])
    #pdb.set_trace()
    if (abs(mean)>MAX_SKEW ): #200e-12 ):
        print('Test Failed in Sample clock: {}MHz, Mean (Average): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
        test_success = False
        #break
    else:
	    print('Test Pass in Sample clock: {}MHz, Means(Avrage): {:.2f} ps , Range:[{:.2f}]ps , Jitter: (deviation): {:.2f}ps'.format(FreqSampleRate,mean/(10**(-12)),(abs(abs(tmax)-abs(tmin)))/(10**(-12)),deviation/(10**(-12))))
    if (highest_mean<abs(mean)):
        highest_mean= abs(mean)
        freq= FreqSampleRate
print("***Skew test completed for channel 1 and chaneel 3***")
print('The Highest mean: {} in freq: {} MHz'.format(highest_mean,freq))

Comparing skew for channel 1 and chanenl 3
Initial Skew Test between 1 to 3 from 1000MHz to 9000MHz in jumps of 1000MHz
Test Failed in Sample clock: 1000MHz, Mean (Average): 165.16 ps , Range:[21.25]ps , Jitter: (deviation): 5.24ps
Test Failed in Sample clock: 2000MHz, Mean (Average): 159.70 ps , Range:[23.89]ps , Jitter: (deviation): 5.31ps
Test Failed in Sample clock: 3000MHz, Mean (Average): 156.58 ps , Range:[22.48]ps , Jitter: (deviation): 5.43ps
Test Failed in Sample clock: 4000MHz, Mean (Average): 155.35 ps , Range:[24.20]ps , Jitter: (deviation): 5.68ps
Test Failed in Sample clock: 5000MHz, Mean (Average): 157.58 ps , Range:[27.96]ps , Jitter: (deviation): 5.10ps
Test Failed in Sample clock: 6000MHz, Mean (Average): 164.52 ps , Range:[24.87]ps , Jitter: (deviation): 5.94ps
Test Failed in Sample clock: 7000MHz, Mean (Average): 159.45 ps , Range:[25.93]ps , Jitter: (deviation): 5.70ps
Test Failed in Sample clock: 8000MHz, Mean (Average): 156.64 ps , Range:[26.72]ps , Jitter: (dev

In [9]:
if (test_success):   
    print('Test Succeed!')
else: 
     print('Test Failed!' )

Test Succeed!


# Disconnect

In [10]:
disconnect()
# Close the connection to the instrument
scope.close()
resourceManager.close()
print('Disconnected')

Disconnected
